In [76]:
from keras.applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, SeparableConvolution2D, GlobalAveragePooling2D ,\
                         Dense, Activation, BatchNormalization
from keras.models import Model
from keras.engine.topology import get_source_inputs
from keras.utils import get_file
from keras.utils import layer_utils
import matplotlib.pyplot as  plt
from keras.preprocessing.image import ImageDataGenerator

In [23]:
def DeepDog(input_tensor=None, input_shape=None, alpha=1, classes=2):

    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                     require_flatten=True)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = Convolution2D(int(32*alpha), (3, 3), strides=(2, 2), padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    x = SeparableConvolution2D(int(32*alpha), (3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    x = SeparableConvolution2D(int(64 * alpha), (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    x = SeparableConvolution2D(int(128 * alpha), (3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    x = SeparableConvolution2D(int(128 * alpha), (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    x = SeparableConvolution2D(int(256 * alpha), (3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    x = SeparableConvolution2D(int(256 * alpha), (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    for _ in range(5):
        x = SeparableConvolution2D(int(512 * alpha), (3, 3), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)

    x = SeparableConvolution2D(int(512 * alpha), (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    x = SeparableConvolution2D(int(1024 * alpha), (3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    
    x = GlobalAveragePooling2D()(x)
    out = Dense(1, activation='sigmoid')(x)

    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, out, name='deepdog')

    return model

In [24]:
model=DeepDog()

In [74]:
model.compile(optimizer='rmsprop',
              loss='mse')

In [120]:
batch_size = 16
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'train/',#This is the target directory\
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'validation/',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')
predict_generator = test_datagen.flow_from_directory(
        'predict/',
        target_size=(224, 224),)


Found 210 images belonging to 1 classes.
Found 169 images belonging to 1 classes.
Found 2 images belonging to 2 classes.


In [112]:
batch_size = 16
epochs=20
model.fit_generator(
        train_generator,
        epochs=20,   
        steps_per_epoch=210 // batch_size,
        validation_data=validation_generator,
        validation_steps=169 // batch_size)

Epoch 1/20
14/13 [================================] - 97s 7s/step - loss: 0.0016 - val_loss: 0.0680
Epoch 2/20
14/13 [================================] - 95s 7s/step - loss: 7.3579e-04 - val_loss: 0.0499
Epoch 3/20
14/13 [================================] - 95s 7s/step - loss: 0.0015 - val_loss: 0.0309
Epoch 4/20
14/13 [================================] - 95s 7s/step - loss: 0.0237 - val_loss: 0.0212
Epoch 5/20
14/13 [================================] - 95s 7s/step - loss: 1.9613e-04 - val_loss: 0.0162
Epoch 6/20
14/13 [================================] - 95s 7s/step - loss: 1.3298e-04 - val_loss: 0.0118
Epoch 7/20
14/13 [================================] - 95s 7s/step - loss: 1.1849e-04 - val_loss: 0.0079
Epoch 8/20
14/13 [================================] - 95s 7s/step - loss: 8.2945e-05 - val_loss: 0.0048
Epoch 9/20
14/13 [================================] - 94s 7s/step - loss: 4.4906e-05 - val_loss: 0.0026
Epoch 10/20
14/13 [================================] - 94s 7s/step - loss: 2

In [121]:
model.predict_generator(generator=predict_generator,steps=1)

array([[ 0.00075852],
       [ 0.00076112]], dtype=float32)

In [ ]:
model.save("DocumentKeras.h5")